In [9]:
from SDP_interaction_inference import simulation
from SDP_interaction_inference.constraints import Constraint
from SDP_interaction_inference import optimization
import numpy as np
import matplotlib.pyplot as plt
from gurobipy import GRB

In [2]:
rng = np.random.default_rng(5)

# Simulation

In [3]:
# simulate
dataset = simulation.simulate_dataset_range_TE([2], rate=5, gene=0.5, tqdm_disable=False)

100%|██████████| 1/1 [00:02<00:00,  2.44s/it]


In [4]:
# downsample
m = 0.5
b = (1 / m) - 1
if b == 0   :
    beta = np.ones(1000)
else:
    beta = rng.beta(1, b, size=1000)
dataset = dataset.downsample(beta)

# Bootstrap

In [21]:
# settings
d = 10

# bootstrap
dataset.bootstrap(d, tqdm_disable=False)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.34s/it]


# Model free

In [6]:
#settings
d = 6
constraints = Constraint(
    moment_bounds=True,
    moment_matrices=True,
    factorization=True
)

# test feasibility
opt_MF = optimization.ModelFreeOptimization(dataset, d, constraints=constraints, printing=False, silent=False)
opt_MF.analyse_dataset()

opt_MF.result_dict

  0%|          | 0/1 [00:00<?, ?it/s]

Set parameter Username
Set parameter LicenseID to value 2616229
Academic license - for non-commercial use only - expires 2026-01-29
Set parameter TimeLimit to value 300
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  300

Optimize a model with 57 rows, 28 columns and 227 nonzeros
Model fingerprint: 0x9005b233
Model has 15 quadratic constraints
Coefficient statistics:
  Matrix range     [1e-04, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 4e+03]
Presolve removed 9 rows and 1 columns
Presolve time: 0.01s

Barrier solved model in 0 iterations and 0.01 seconds (0.00 work units)
Model is infeasible


100%|██████████| 1/1 [00:00<00:00,  3.72it/s]


{0: {'status': 'INFEASIBLE', 'time': 0.01100015640258789, 'cuts': 0}}

# Birth-Death

In [7]:
#settings
d = 4
constraints = Constraint(
    moment_bounds=True,
    moment_matrices=True,
    moment_equations=True,
    factorization=False
)
fixed = [(1, 1), (3, 1)]

# test feasibility
opt = optimization.BirthDeathOptimization(dataset, d, constraints=constraints, silent=False) #fixed=fixed)
opt.analyse_dataset()

opt.result_dict

  0%|          | 0/1 [00:00<?, ?it/s]

Set parameter Username
Set parameter LicenseID to value 2616229
Academic license - for non-commercial use only - expires 2026-01-29
Set parameter TimeLimit to value 300
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  300

Optimize a model with 32 rows, 20 columns and 74 nonzeros
Model fingerprint: 0x5ab7de89
Model has 10 quadratic constraints
Coefficient statistics:
  Matrix range     [6e-04, 1e+00]
  QMatrix range    [1e+00, 3e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+02, 1e+02]
  RHS range        [1e-01, 8e+01]
Presolve removed 23 rows and 4 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 14 rows and 4 columns
Presolve time: 0.00s
Presolved: 118 rows, 39 columns, 354 nonzeros
Presolved model has 23 

100%|██████████| 1/1 [00:00<00:00,  7.19it/s]


{0: {'status': 'OPTIMAL', 'time': 0.023000001907348633, 'cuts': 0}}

# Telegraph

In [22]:
#settings
d = 7
constraints = Constraint(
    moment_bounds=True,
    moment_matrices=True,
    moment_equations=True,
    telegraph_moments=True,
    telegraph_moments_ineq=True,
    telegraph_factorization=False
)
fixed = [(3, 1), (7, 1)]

# test feasibility
opt_TE = optimization.TelegraphOptimization(dataset, d, constraints=constraints, write_model=True, silent=True)#, fixed=fixed)
opt_TE.analyse_dataset()

opt_TE.result_dict

100%|██████████| 1/1 [05:30<00:00, 330.80s/it]


{0: {'status': 'TOTAL_TIME_LIMIT', 'time': 326.0920000076294, 'cuts': 11}}

# General model

In [ ]:
#settings
reactions = [
    "1",
    "xs[0]",
    "1",
    "xs[1]",
    "xs[0] * xs[1]"
]
vrs = [
    [1, 0],
    [-1, 0],
    [0, 1],
    [0, -1],
    [-1, -1]
]
db = 2
R = 5
S = 2
U = []
constraints = Constraint(
    moment_bounds=True,
    moment_matrices=True,
    moment_equations=True
)
fixed = [(1, 1)]
d = 6

# test feasibility
opt_G = optimization.Optimization(dataset, constraints, reactions, vrs, db, R, S, U, d, fixed, printing=False, silent=False)
opt_G.analyse_dataset()

opt_G.result_dict

  0%|          | 0/1 [00:00<?, ?it/s]

Set parameter Username
Set parameter LicenseID to value 2616229
Academic license - for non-commercial use only - expires 2026-01-29
Set parameter TimeLimit to value 300
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  300

Optimize a model with 58 rows, 33 columns and 228 nonzeros
Model fingerprint: 0xe83e2e6d
Model has 21 quadratic constraints
Coefficient statistics:
  Matrix range     [1e-04, 1e+00]
  QMatrix range    [1e+00, 1e+01]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+02, 1e+02]
  RHS range        [1e-01, 4e+03]
Presolve removed 36 rows and 4 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 14 rows and 4 columns
Presolve time: 0.00s
Presolved: 299 rows, 88 columns, 1040 nonzeros
Presolved model has 5

100%|██████████| 1/1 [00:01<00:00,  1.16s/it]


{0: {'status': 'OPTIMAL', 'time': 0.25999975204467773, 'cuts': 6}}